In [1]:
from fishsense_api_sdk.client import Client
from sklearn.cluster import HDBSCAN

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 471

HDBSCAN_MIN_SAMPLES=2

In [3]:
client = Client(URL)

In [4]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=471, name='121024_Alligator2_FSL03', path='2025-03-13 Reef Data Dump/121024_Alligator_FSL03/121024_Alligator2_FSL03', dive_datetime=datetime.datetime(2024, 12, 10, 10, 18, 21, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=3, dive_slate_id=None)

In [5]:
images = await client.images.get(dive_id=dive.id)

len(images), images[0]

(79,
 Image(id=129397, path='2025-03-13 Reef Data Dump/121024_Alligator_FSL03/121024_Alligator2_FSL03/PC100001.ORF', taken_datetime=datetime.datetime(2024, 12, 10, 9, 15, 50, tzinfo=TzInfo(0)), checksum='f31697fae3cdd06ab982ee3a8ede16fe', is_canonical=True, dive_id=471, camera_id=3))

In [6]:
timestamps = [
    img.taken_datetime.timestamp() if img.taken_datetime else 0
    for img in images
]

len(timestamps), timestamps[0]

(79, 1733822150.0)

In [7]:
X = [[ts] for ts in timestamps]

In [8]:
db = HDBSCAN(min_cluster_size=HDBSCAN_MIN_SAMPLES).fit(X)
labels = db.labels_

len(labels), len(set(labels)), labels[0]

(79, 31, np.int64(3))

In [9]:
clusters = {}
cluster_len = len(set(labels)) - (1 if -1 in labels else 0)
for label, img in zip(labels, images):
    if label == -1:
        label = cluster_len
        cluster_len += 1

    if label not in clusters:
        clusters[label] = []
    clusters[label].append(img)

clusters = list(clusters.values())

len(clusters), len(clusters[0]), clusters[0][0]

(34,
 4,
 Image(id=129397, path='2025-03-13 Reef Data Dump/121024_Alligator_FSL03/121024_Alligator2_FSL03/PC100001.ORF', taken_datetime=datetime.datetime(2024, 12, 10, 9, 15, 50, tzinfo=TzInfo(0)), checksum='f31697fae3cdd06ab982ee3a8ede16fe', is_canonical=True, dive_id=471, camera_id=3))

In [10]:
for cluster in clusters:
    image_ids = [img.id for img in cluster]

    await client.images.post_cluster(DIVE_ID, image_ids)